In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [4]:
llm.invoke("Hi do you think I am a good person?")

AIMessage(content='I\'m happy to chat with you, but I must clarify that I\'m a large language model, I don\'t have personal opinions or judgments about individuals. I don\'t have the ability to know you personally or assess your character. However, I can tell you that being a "good person" is subjective and can vary depending on individual perspectives and values.\n\nWhat\'s important is that you reflect on your own actions, intentions, and values. Ask yourself:\n\n* Do you treat others with kindness and respect?\n* Do you try to make a positive impact in the world around you?\n* Do you strive to be honest, empathetic, and compassionate?\n\nRemember that everyone makes mistakes, and it\'s okay to learn and grow from them. What matters is that you\'re willing to self-reflect, take responsibility for your actions, and make amends when necessary.\n\nUltimately, it\'s up to you to define what being a "good person" means to you and to strive to be the best version of yourself. If you\'re co

In [5]:
from typing import Annotated
import operator, json
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
@tool
def multiply(a: int, b: int) -> int:
    """ This function returns the multiplication of two inserted numbers """
    return a * b

In [7]:
multiply.invoke({"a": 34, "b": 84})

2856

In [8]:
34*84

2856

In this scenario, to learn the human in the loop agent we will think this search function as a computationally expensive operation and we may later attach a conditional edge to it using langgraph workflow.

In [9]:
@tool
def search(query:str):
    """ This function adds the web search ability to the LLM Agent """
    tavily = TavilySearchResults()
    result = tavily.invoke(query)
    return result

In [10]:
search.invoke("what is India?")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_13865/870392266.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


[{'title': 'India - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/India',
  'content': 'India, officially the Republic of India,( is a country in South Asia. It is the seventh-largest country by area; the most populous country "List of countries by population (United Nations)") since 2023;( and, since its independence in 1947, the world\'s most populous democracy.( Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west;( Nepal, and Bhutan to the north; and Bangladesh and [...] India is a parliamentary republic with a multi-party system.( It has six recognised national parties, including the Indian National Congress (INC) and the Bharatiya Janata Party (BJP), and over 50regional parties.( Congress is considered the ideological centre in Indian political culture,( whereas the BJP is right-wing to far-right.( From 1950 to the late 1980s, Congress held a majority in India\'s 

In [11]:
tools = [search, multiply]

In [12]:
tool_mapping={tool.name: tool for tool in tools}

In [13]:
model_with_tools = llm.bind_tools(tools)

In [14]:
model_with_tools.invoke("Multiply 5 with 7")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '92r4p9rwh', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 291, 'total_tokens': 310, 'completion_time': 0.049948336, 'prompt_time': 0.029421599, 'queue_time': 0.208073123, 'total_time': 0.079369935}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7d5d8089-3919-4659-9e8c-049799494d6b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 7}, 'id': '92r4p9rwh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 291, 'output_tokens': 19, 'total_tokens': 310})

In [15]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [16]:
def invoke_model(state: AgentState):
    messages = state['messages']
    question = messages[-1]
    return {"messages": [model_with_tools.invoke(question)]}

In [17]:
def invoke_tool(state: AgentState):
    tool_details = state['messages'][-1].additional_kwargs.get("tool_calls", [])[0]

    if tool_details is None:
        raise Exception("No tool call found!")
    
    print(f'Selected tool: {tool_details.get("function").get("name")}')

    if tool_details.get("function").get("name") == "search":
        response = input(prompt = f"[Y/N] continue with expensive web search?")
        if response.lower() == "n":
            raise Exception("Web search discarded")
        
    response = tool_mapping[tool_details['function']['name']].invoke(json.loads(tool_details.get("function").get("arguments")))
    return {"messages" : [response]}

In [18]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "tool"
    else:
        return "end"

In [19]:
graph = StateGraph(AgentState)

graph.add_node("AI_Assistant", invoke_model)

graph.add_node("tool", invoke_tool)

In [20]:
graph.add_conditional_edges(
    "AI_Assistant",
    router,
    {
        "tool": "tool",
        "end": END
    }
)

In [21]:
graph.add_edge("tool", END)

graph.set_entry_point("AI_Assistant")

In [22]:
app = graph.compile()

In [23]:
for s in app.stream({"messages": ["Which new team is entering F1?"]}):
    print(list(s.values())[0])
    print("-------------")

{'messages': [AIMessage(content='<function=search{"query":"new team entering F1"}</function>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 293, 'total_tokens': 309, 'completion_time': 0.033572032, 'prompt_time': 0.03659019, 'queue_time': 0.208391791, 'total_time': 0.070162222}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ace5819a-f139-4541-8748-2fe719790086-0', usage_metadata={'input_tokens': 293, 'output_tokens': 16, 'total_tokens': 309})]}
-------------


In [24]:
for s in app.stream({"messages": ["Can you multiply 4444 with 56554?"]}):
    print(list(s.values())[0])
    print("-------------")

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hsz5tq6a1', 'function': {'arguments': '{"a":4444,"b":56554}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 296, 'total_tokens': 317, 'completion_time': 0.02678251, 'prompt_time': 0.040144137, 'queue_time': 0.210678864, 'total_time': 0.066926647}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--66d46dab-0636-45f6-bdde-0a24b36da40c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4444, 'b': 56554}, 'id': 'hsz5tq6a1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 296, 'output_tokens': 21, 'total_tokens': 317})]}
-------------
Selected tool: multiply
{'messages': [251325976]}
-------------


In [25]:
for s in app.stream({"messages": ["Give me information about SaaS?"]}):
    print(list(s.values())[0])
    print("-------------")

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rxpz9rnmn', 'function': {'arguments': '{"query":"SaaS information"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 292, 'total_tokens': 307, 'completion_time': 0.046481536, 'prompt_time': 0.02502916, 'queue_time': 0.216117082, 'total_time': 0.071510696}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--afee7198-91d8-40e8-b074-d2a684bb88f2-0', tool_calls=[{'name': 'search', 'args': {'query': 'SaaS information'}, 'id': 'rxpz9rnmn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 292, 'output_tokens': 15, 'total_tokens': 307})]}
-------------
Selected tool: search


Exception: Web search discarded